In [4]:
!conda install -c conda-forge geopy --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
!pip install beautifulsoup4 

In [5]:
!pip install geocoder

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder#import geocoder
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


In [7]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
# create a new DataFrame from the list
Mu_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Mu_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [8]:
Mu_df.shape

(42, 1)

In [9]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in Mu_df["Neighborhood"].tolist() ]

In [11]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00426978951916, 72.85579215153318],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003

In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
Mu_df['Latitude'] = df_coords['Latitude']
Mu_df['Longitude'] = df_coords['Longitude']
# check the neighborhoods and the coordinates
print(Mu_df.shape)
Mu_df

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [13]:
Mu_df.to_csv("Mu_df.csv", index=False)

# Create a map of Mumbai with neighborhoods superimposed on top

In [14]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India is 18.9387711, 72.8353355.


In [15]:
map_Mu = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Mu_df['Latitude'], Mu_df['Longitude'], Mu_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Mu)  
    
map_Mu

In [16]:
map_Mu.save('map_Mu.html')

# Use the Foursquare API to explore the neighborhoods

In [18]:
CLIENT_ID = 'ZTL5UQ1TEI4HU0WEXGPNZ33GEXKU0J50K15MYWXAFX3RAWIX' # your Foursquare ID
CLIENT_SECRET = 'DWZQXXI2RQJFTNJE3ZCTGXGDRMDOX1AMXKXUVV5AYRYLGIIK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZTL5UQ1TEI4HU0WEXGPNZ33GEXKU0J50K15MYWXAFX3RAWIX
CLIENT_SECRET:DWZQXXI2RQJFTNJE3ZCTGXGDRMDOX1AMXKXUVV5AYRYLGIIK


In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Mu_df['Latitude'], Mu_df['Longitude'], Mu_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2714, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


# analyze each neighbourhood

In [21]:
Mu_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Mu_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Mu_onehot.columns[-1]] + list(Mu_onehot.columns[:-1])
Mu_onehot = Mu_onehot[fixed_columns]

print(Mu_onehot.shape)
Mu_onehot.head()

(2714, 200)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
Mu_grouped = Mu_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Mu_grouped.shape)
Mu_grouped

(42, 200)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.0,0.000000,0.000000,0.010000,0.000000,0.0,0.020000,0.010000,0.010000,0.020000,0.000000,0.060000,0.00,0.0000,0.00,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.0000,0.000000,0.000000,0.050000,0.00,0.050000,0.000000,0.03,0.020000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.00000,0.000000,0.010000,0.00,0.000000,0.020000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.040000,0.150000,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.010000,0.020000,0.010000,0.000000,0.01,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.010000,0.00,0.000000,0.000000,0.00,0.000000,0.00000,0.00000,0.00,0.00,0.000000,0.000000,0

In [23]:
len(Mu_grouped[Mu_grouped["Shopping Mall"] > 0])

18

In [24]:
Mu_mall = Mu_grouped[["Neighborhoods","Shopping Mall"]]
Mu_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.00
1,Anushakti Nagar,0.00
2,Baiganwadi,0.00
3,Bandra,0.00
4,Bhandup,0.04


# Cluster Neighborhoods

In [25]:
kclusters = 3

Mu_clustering = Mu_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mu_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 2, 0, 2, 0, 2])

In [26]:
Mu_merged = Mu_mall.copy()

# add clustering labels
Mu_merged["Cluster Labels"] = kmeans.labels_
Mu_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Mu_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.00,0
1,Anushakti Nagar,0.00,0
2,Baiganwadi,0.00,0
3,Bandra,0.00,0
4,Bhandup,0.04,1


In [27]:
Mu_merged = Mu_merged.join(Mu_df.set_index("Neighborhood"), on="Neighborhood")

print(Mu_merged.shape)
Mu_merged.head()

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.00,0,19.118459,72.841763
1,Anushakti Nagar,0.00,0,19.042830,72.927340
2,Baiganwadi,0.00,0,19.062940,72.926630
3,Bandra,0.00,0,19.054370,72.840170
4,Bhandup,0.04,1,19.145560,72.948560


In [28]:
print(Mu_merged.shape)
Mu_merged.sort_values(["Cluster Labels"], inplace=True)
Mu_merged

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
35,Tilak Nagar (Mumbai),0.000000,0,18.996160,72.852790
32,"Sion, Mumbai",0.000000,0,19.043410,72.863320
31,Shil Phata,0.000000,0,19.146580,73.040050
30,Seven Bungalows,0.000000,0,19.131342,72.816342
28,Mumbra,0.000000,0,19.167632,73.021408
26,Mogra Village,0.000000,0,24.375900,75.954570
25,Mira Road,0.000000,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.000000,0,18.950694,72.827268
23,Mankhurd,0.000000,0,19.048530,72.932220


In [29]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mu_merged['Latitude'], Mu_merged['Longitude'], Mu_merged['Neighborhood'], Mu_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
map_clusters.save('map_clusters.html')

In [31]:
Mu_merged.loc[Mu_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
35,Tilak Nagar (Mumbai),0.0,0,18.996160,72.852790
32,"Sion, Mumbai",0.0,0,19.043410,72.863320
31,Shil Phata,0.0,0,19.146580,73.040050
30,Seven Bungalows,0.0,0,19.131342,72.816342
28,Mumbra,0.0,0,19.167632,73.021408
26,Mogra Village,0.0,0,24.375900,75.954570
25,Mira Road,0.0,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.0,0,18.950694,72.827268
23,Mankhurd,0.0,0,19.048530,72.932220


In [32]:

Mu_merged.loc[Mu_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
10,Dombivli,0.047619,1,19.21275,73.08324
36,Uttan,0.050000,1,26.86634,80.93884
19,Kanjurmarg,0.040000,1,19.13138,72.93568
38,Vikhroli,0.030000,1,19.11109,72.92781
27,Mulund,0.037037,1,19.17183,72.95565
34,Thakur village,0.048387,1,19.21020,72.87541
4,Bhandup,0.040000,1,19.14556,72.94856


In [33]:

Mu_merged.loc[Mu_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
37,Vashi,0.020000,2,19.084650,72.904810
39,Wadala,0.020000,2,19.017200,72.858170
5,Borivali,0.010526,2,19.229360,72.857510
29,Pestom sagar,0.012048,2,19.070640,72.902170
7,Chembur,0.010101,2,19.062180,72.902410
9,Devipada,0.011236,2,19.224690,72.866050
40,Western Suburbs (Mumbai),0.012500,2,19.197010,72.827680
12,Ghatkopar,0.012048,2,19.086523,72.909008
13,Goregaon,0.020000,2,19.164550,72.849460
33,"Sonapur, Bhandup",0.023810,2,19.163940,72.935440


# Most of the shopping malls are concentrated in the central and east area of Mumbai city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central and east area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition